In [14]:
# install bonita first
!pip install -e git+https://github.com/BatsResearch/bonito#egg=bonito

error: Failed to parse `git+https://github.com/BatsResearch/bonito`
  Caused by: Unsupported URL (expected a `file://` scheme): `git+https://github.com/BatsResearch/bonito`


We can start by downloading all our documentation from our Gitbook docs located
at docs.zenml.io. We'll use the langchain scraper to make this easy.

In [2]:
from zenml.client import Client

artifact = Client().get_artifact_version('fb39d530-6655-4cc4-a2dd-a3a85af973e2')
loaded_artifact = artifact.load()

In [6]:
from rich import inspect

inspect(loaded_artifact[0])

╭─────────────────────────────── <class 'langchain_core.documents.base.Document'> ────────────────────────────────╮
│ Class for storing a piece of text and associated metadata.                                                      │
│                                                                                                                 │
│ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ Document(                                                                                                   │ │
│ │ │   page_content='GitHub Container Registry\n\nStoring container images in GitHu'+1721,                     │ │
│ │ │   metadata={'source': 'https://docs.zenml.io/stacks-and-components/component-guide/'+27}                  │ │
│ │ )                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                                 │
│ lc_attributes = {}                                                                                              │
│    lc_secrets = {}                                                                                              │
│      metadata = {                                                                                               │
│                     'source':                                                                                   │
│                 'https://docs.zenml.io/stacks-and-components/component-guide/container-registries/github'       │
│                 }                                                                                               │
│  page_content = "GitHub Container Registry\n\nStoring container images in GitHub.\n\nThe GitHub container       │
│                 registry is a\n\ncontainer registry\n\nflavor that comes built-in with ZenML and uses           │
│                 the\n\nGitHub Container Registry\n\nto store container images.\n\nWhen to use it\n\nYou should  │
│                 use the GitHub container registry if:\n\none or more components of your stack need to pull or   │
│                 push container images.\n\nyou're using GitHub for your projects. If you're not using GitHub,    │
│                 take a look at the other container registry flavors.\n\nHow to deploy it\n\nThe GitHub          │
│                 container registry is enabled by default when you create a GitHub account.\n\nHow to find the   │
│                 registry URI\n\nThe GitHub container registry URI should have the following                     │
│                 format:\n\nghcr.io/<USER_OR_ORGANIZATION_NAME>\n\n#                                             │
│                 Examples:\n\nghcr.io/zenml\n\nghcr.io/my-username\n\nghcr.io/my-organization\n\nTo figure our   │
│                 the URI for your registry:\n\nUse the GitHub user or organization name to fill the template     │
│                 ghcr.io/<USER_OR_ORGANIZATION_NAME> and get your URI.\n\nHow to use it\n\nTo use the GitHub     │
│                 container registry, we need:\n\n\u200bDocker installed and running.\n\nThe registry URI. Check  │
│                 out the previous section on the URI format and how to get the URI for your registry.\n\nOur     │
│                 Docker client configured, so it can pull and push images. Follow this guide to create a         │
│                 personal access token and login to the container registry.\n\nWe can then register the          │
│                 container registry and use it in our active stack:\n\nzenml container-registry                  │
│                 register\n\nNAME\n\n--flavor=github \\\n\n--uri=<REGISTRY_URI>\n\n# Add the container registry  │
│                 to the active stack\n\nzenml stack upd

In [8]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 8.2 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [9]:
import spacy

nlp = spacy.load("en_core_web_sm")  # Load the English language model

def split_into_sentences(text):
    doc = nlp(text)  # Process the text with SpaCy
    sentences = [sent.text.strip() for sent in doc.sents]  # Extract sentences and strip whitespace
    return sentences

def process_documents(documents):
    all_sentences = []
    for doc in documents:
        text = doc.page_content  # Extract the text from the Document object
        sentences = split_into_sentences(text)  # Split the text into sentences
        all_sentences.extend(sentences)  # Add the sentences to the list
    return all_sentences

  # Your list of langchain Document objects
sentences = process_documents(loaded_artifact)  # Process the documents and get all sentences
sentences[0:5]

['GitHub Container Registry\n\nStoring container images in GitHub.',
 'The GitHub container registry is a\n\ncontainer registry\n\nflavor that comes built-in with ZenML and uses the\n\nGitHub Container Registry\n\nto store container images.',
 'When to use it\n\nYou should use the GitHub container registry if:\n\none or more components of your stack need to pull or push container images.',
 "you're using GitHub for your projects.",
 "If you're not using GitHub, take a look at the other container registry flavors."]

In [10]:
from datasets import Dataset

# Assuming sentences is a list of strings, where each string is a sentence
data = {"sentence": sentences}
dataset = Dataset.from_dict(data)

print(dataset)

Dataset({
    features: ['sentence'],
    num_rows: 8190
})


In [11]:
from bonito import Bonito, SamplingParams
from datasets import load_dataset

# Initialize the Bonito model
bonito = Bonito("BatsResearch/bonito-v1")

sampling_params = SamplingParams(max_tokens=256, top_p=0.95, temperature=0.5, n=1)
synthetic_dataset = bonito.generate_tasks(
    dataset,
    context_col="sentence",
    task_type="qg",
    sampling_params=sampling_params
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 from bonito import Bonito, SamplingParams                                                   │
│    2 from datasets import load_dataset                                                           │
│    3                                                                                             │
│    4 # Initialize the Bonito model                                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'bonito'